In [1]:
from sklearn import preprocessing
import numpy as np


x_train = np.array([[1, -1, 2], [2, 0, 0], [0, 1, -1]], dtype=np.float64)
x_test = np.array([[-1, 1, 0]], dtype=np.float64)

# facem statisticile pe datele de antrenare
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)

# afisam media
print(scaler.mean_) 		 	# => [1.  0.  0.33333333]
# afisam deviatia standard
print(scaler.scale_)			# => [0.81649658 0.81649658 1.24721913]

# scalam datele de antrenare
scaled_x_train = scaler.transform(x_train)
print(scaled_x_train) 	# => [[0.          -1.22474487  1.33630621]
                             #     [1.22474487   0.          -0.26726124]
                             #     [-1.22474487  1.22474487  -1.06904497]]

# scalam datele de test
scaled_x_test = scaler.transform(x_test)
print(scaled_x_test)        # => [[-2.44948974  1.22474487 -0.26726124]]


[1.         0.         0.33333333]
[0.81649658 0.81649658 1.24721913]
[[ 0.         -1.22474487  1.33630621]
 [ 1.22474487  0.         -0.26726124]
 [-1.22474487  1.22474487 -1.06904497]]
[[-2.44948974  1.22474487 -0.26726124]]


In [30]:
#2

def normalize_data(train_data, test_data, type1=None):
    scaler = None

    if type1 == 'standard':
        scaler = preprocessing.StandardScaler()
        scaler.fit(train_data)
        
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data)
        
        return scaled_train_data, scaled_test_data
    
    elif type1 == 'l1':
        scaler = preprocessing.Normalizer(norm='l1')
        scaler.fit(train_data)
        
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data)
        
        return scaled_train_data, scaled_test_data
    
    elif type1 == 'l2':
        scaler = preprocessing.Normalizer(norm='l2')
        scaler.fit(train_data)
        
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data)
        
        return scaled_train_data, scaled_test_data


In [31]:
#3

class BagOfWords:
    def __init__(self):
        self.vocabulary = {}
        self.words = []
        self.vocabulary_len = 0
        
        
    def build_vocabulary(self, data):
        for document in data:
            for word in document:
                if word not in self.vocabulary:
                    self.vocabulary[word] = len(self.vocabulary)
                    self.words.append(word)
                    
            self.vocabulary_len = len(self.vocabulary)
            
#4
    def get_features(self, data):
        features = np.zeros((len(data), self.vocabulary_len))
        
        for id_sen, document in enumerate(data):
            for word in document:
                if word in self.vocabulary:
                    features[id_sen, self.vocabulary[word]] += 1
                    
        return features

In [3]:
#5
train_sentences = np.load("data/training_sentences.npy", allow_pickle=True)
train_labels = np.load("data/training_labels.npy", allow_pickle=True)

test_sentences = np.load("data/test_sentences.npy", allow_pickle=True)
test_labels = np.load("data/test_labels.npy", allow_pickle=True)

In [32]:
bow = BagOfWords()
bow.build_vocabulary(train_sentences)

train_features = bow.get_features(train_sentences)
test_features = bow.get_features(test_sentences)

train_features_norm, test_features_norm = normalize_data(train_features, test_features, type1='l2')

print(test_features_norm)

[[0.  0.  0.5 ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]


In [33]:
#6
from sklearn import svm

svm_model = svm.SVC(C=1, kernel='linear')
svm_model.fit(train_features_norm, train_labels)

SVC(C=1, kernel='linear')

In [37]:
predicted = svm_model.predict(test_features_norm)

correct = 0
for idx, elem in enumerate(predicted):
    if elem == test_labels[idx]:
        correct += 1
        
proccent = correct / len(predicted)
print(proccent)

0.9842391304347826


In [39]:
from sklearn.metrics import f1_score

score_f1 = f1_score(y_true=test_labels,y_pred=predicted)
print(score_f1)

0.9409368635437881


In [47]:
coefs = svm_model.coef_

indexes = np.argsort(coefs)
worst = np.array(list(bow.vocabulary.keys())[indexes[:10]])

print(worst)

TypeError: only integer scalar arrays can be converted to a scalar index